In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#Extracting the data and formatting it in a way convenient to us
data1=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NLP Spam Classifier/spam.csv',encoding='latin-1')
data1.head()
data1 = data1[['v2', 'v1']]
data1=data1.rename(columns={"v1":"Type","v2":"Mail"})
data1.head()
 





,Mail,Type
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


In [ ]:
cleanmails = []
porterStemmer=PorterStemmer()
for i in range(0,len(data1)):
  review=data1['Mail'][i]
  review=review.lower()
  review=re.sub(r'[^0-9a-zA-Z]',' ',review)
  review=review.split()
  review=" ".join([porterStemmer.stem(word) for word in review if word not in stopwords.words('english')])
  cleanmails.append(review)

#print(cleanmails)


In [ ]:
X=cleanmails
Y=data1['Type']


from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

def solve(model, X, y):
    # train test split
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, shuffle=True, stratify=y)
    # model training
    pipeline_model = Pipeline([('vect', CountVectorizer()),
                              ('tfidf', TfidfTransformer()),
                              ('clf', model)])
    pipeline_model.fit(x_train, y_train)
    
    print('Accuracy:', pipeline_model.score(x_test, y_test)*100)
    y_pred = pipeline_model.predict(x_test)
    print(classification_report(y_test, y_pred))

In [13]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
solve(model, X, Y)

Accuracy: 97.05671213208902
              precision    recall  f1-score   support

         ham       0.97      1.00      0.98      1206
        spam       0.99      0.79      0.88       187

    accuracy                           0.97      1393
   macro avg       0.98      0.89      0.93      1393
weighted avg       0.97      0.97      0.97      1393



In [12]:
from sklearn.svm import SVC
model = SVC(C=2)
solve(model, X, Y)


Accuracy: 98.34888729361091
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99      1206
        spam       1.00      0.88      0.93       187

    accuracy                           0.98      1393
   macro avg       0.99      0.94      0.96      1393
weighted avg       0.98      0.98      0.98      1393



In [11]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
solve(model, X, Y)

Accuracy: 97.48743718592965
              precision    recall  f1-score   support

         ham       0.97      1.00      0.99      1206
        spam       1.00      0.81      0.90       187

    accuracy                           0.97      1393
   macro avg       0.99      0.91      0.94      1393
weighted avg       0.98      0.97      0.97      1393

